In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from textblob import Word
from textblob import TextBlob
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from modAL.models import ActiveLearner
from modAL.models import ActiveLearner
import en_core_web_sm
nlp = en_core_web_sm.load()

%matplotlib inline
from scipy import spatial


In [3]:
def word_embed(word):
    try:
        vec = model[word]
        vec = vec.reshape(1,vec.shape[0])
    except:
        vec = np.zeros((1, 300))
        #this is hardcoded
    return vec

def model_embed_ref(data):
    sentence  = data ['ref_demoted']
    sentence_array = [word_embed(word) for word in sentence.split()]
    return np.sum(sentence_array,axis=0)

def model_embed_stud(data):
    sentence  = data ['student_demoted']
    sentence_array = [word_embed(word) for word in sentence.split()]
    return np.sum(sentence_array,axis=0)

def cos_similarity_demo(data):
    return 1 - spatial.distance.cosine(data['embed_ref_demoted'],data['embed_stud_demoted'])

def cos_similarity(data):
    return 1 - spatial.distance.cosine(data['embed_ref'],data['embed_stud'])

def align_ratio(data):
    return (2*len(data['aligned'])) / (len(data['ref_answer'].split()) + len(data['student_answer'].split()))

def align_ratio_demo(data):
    if (len(data['ref_demoted'].split()) + len(data['student_demoted'].split())) == 0:
        return 0.0
    else:
        return (2*len(data['aligned_demoted'])) / (len(data['ref_demoted'].split()) + len(data['student_demoted'].split()))

In [4]:
df = pd.read_pickle("./feature_saved_demoted.pkl") 
df2 = pd.read_pickle("./feature_demoted_embed.pkl") 
df[['embed_ref_demoted','embed_stud_demoted']] = df2[['embed_ref_demoted','embed_stud_demoted']]
df

,question_id,question,ref_answer,student_answer,grade,qn_modified,ref_modified,student_modified,student_demoted,ref_demoted,length_ratio,grades_round,embed_ref,embed_stud,aligned,aligned_demoted,embed_ref_demoted,embed_stud_demoted
0,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,High risk problems are address in the prototyp...,3.5,what role prototype program in problem solving,to simulate behaviour portion desired software...,high risk problem address in prototype program...,high risk address make sure that feasible may ...,simulate behaviour portion desired software pr...,3.055556,4,"[[0.13071777343750002, -0.10377929687499998, 0...","[[-0.5155224609375, 0.008998413085937518, 1.30...","[[software, software]]","[[software, software]]","[[0.12071777343750001, -0.11377929687499999, 0...","[[-0.42614746, -0.44728088, 1.0943298, 1.98028..."
1,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To simulate portions of the desired final prod...,5.0,what role prototype program in problem solving,to simulate behaviour portion desired software...,to simulate portion desired final product quic...,simulate portion desired final product quick a...,simulate behaviour portion desired software pr...,3.074074,5,"[[0.13071777343750002, -0.10377929687499998, 0...","[[2.15602783203125, 0.4706297302246094, 0.2766...","[[to, to], [simulate, simulate], [portion, por...","[[portion, portion], [desired, desired], [simu...","[[0.12071777343750001, -0.11377929687499999, 0...","[[1.87479736328125, 0.3003001403808594, -0.136..."
2,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,A prototype program simulates the behaviors of...,4.0,what role prototype program in problem solving,to simulate behaviour portion desired software...,prototype program simulates behavior portion d...,simulates behavior portion desired software pr...,simulate behaviour portion desired software pr...,1.722222,4,"[[0.13071777343750002, -0.10377929687499998, 0...","[[-0.28229736328125, 0.043576965332031245, 0.2...","[[portion, portion], [desired, desired], [soft...","[[portion, portion], [desired, desired], [soft...","[[0.12071777343750001, -0.11377929687499999, 0...","[[-0.1350708, -0.15441132, 0.1538086, 0.227294..."
3,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,Defined in the Specification phase a prototype...,5.0,what role prototype program in problem solving,to simulate behaviour portion desired software...,defined in specification phase prototype stimu...,defined specification phase stimulates behavio...,simulate behaviour portion desired software pr...,3.814815,5,"[[0.13071777343750002, -0.10377929687499998, 0...","[[-0.127725830078125, 0.53630615234375, 0.3706...","[[portion, portion], [desired, desired], [soft...","[[portion, portion], [desired, desired], [soft...","[[0.12071777343750001, -0.11377929687499999, 0...","[[-0.3446045, -0.018371582, 0.34469604, 1.6362..."
4,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,It is used to let the users have a first idea ...,3.0,what role prototype program in problem solving,to simulate behaviour portion desired software...,it used to let user first idea completed progr...,it used let user first idea completed and allo...,simulate behaviour portion desired software pr...,3.462963,3,"[[0.13071777343750002, -0.10377929687499998, 0...","[[0.23586181640625004, 0.06217674255371097, 0....","[[software, software], [product, generate]]","[[software, software], [product, generate]]","[[0.12071777343750001, -0.11377929687499999, 0...","[[0.38285400390625, -0.15020606994628904, 0.43..."
5,1.1,What is the role of a prototype program in pro...,To simulate the behaviour of portions of the d...,To find problem and errors in a program before...,2.0,what role prototype prog

In [6]:
df['cos_similarity'] = df.apply(cos_similarity, axis=1)
df['cos_similarity_demo'] = df.apply(cos_similarity_demo, axis=1)

#getting aligned scores
df['aligned_score'] = df.apply(align_ratio, axis=1)
df['aligned_score_demo'] = df.apply(align_ratio_demo, axis=1)

/home/mohandass/anaconda3/envs/maluuba/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [7]:
df
df.to_pickle("./sultan_features.pkl", protocol=4)

In [8]:
df[['aligned_demoted','ref_demoted','student_demoted']]

,aligned_demoted,ref_demoted,student_demoted
0,"[[software, software]]",simulate behaviour portion desired software pr...,high risk address make sure that feasible may ...
1,"[[portion, portion], [desired, desired], [simu...",simulate behaviour portion desired software pr...,simulate portion desired final product quick a...
2,"[[portion, portion], [desired, desired], [soft...",simulate behaviour portion desired software pr...,simulates behavior portion desired software pr...
3,"[[portion, portion], [desired, desired], [soft...",simulate behaviour portion desired software pr...,defined specification phase stimulates behavio...
4,"[[software, software], [product, generate]]",simulate behaviour portion desired software pr...,it used let user first idea completed and allo...
5,[],simulate behaviour portion desired software pr...,find and error before it finalized
6,[],simulate behaviour portion desired software pr...,address major issue creation there no way acco...
7,"[[simulate, simulate], [portion, part]]",simulate behaviour portion desired software pr...,you can break whole into simulate part final
8,[],simulate behaviour portion desired software pr...,provide example or model how finished should p...
9,"[[product, product], [portion, portion], [desi...",simulate behaviour portion desired software pr...,simulating behavior only portion desired softw...
